# Evaluation

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


benchmarks = pd.read_csv("benchmarks.csv")
benchmarks

,test,tool,bytes_before,bytes_after,nodes_before,nodes_after,err_nodes_before,err_nodes_after,time_sec,predicate_calls,failed_compiles,failed_runs,other
0,test01,creduce,93,27,48,15,0,1,6.67,812,0,62,NaN
1,test01,perses,93,33,48,22,0,0,1.45,19,0,7,NaN
2,test01,bric-ddmin,93,93,48,48,0,0,0.05,2,0,0,NaN
3,test01,bric-hdd,93,52,48,1,0,1,3.66,373,0,21,NaN
4,test01,bric-br,93,93,48,48,0,0,0.03,1,0,0,NaN
5,test01,bric-gbr,93,93,48,48,0,0,0.03,1,0,0,NaN
6,test02,creduce,160,30,72,15,0,1,8.62,991,0,70,NaN
7,test02,perses,160,72,72,39,0,0,1.80,34,0,13,NaN
8,test02,bric-ddmin,160,160,72,72,0,0,0.08,3,0,0,NaN
9,test02,bric-hdd,160,92,72,37,0,2,6.57,662,0,38,NaN
